In [26]:
# --- 1. Install dependencies ---
!pip install geopandas folium pandas shapely requests statsmodels matplotlib

import geopandas as gpd
import pandas as pd
import folium
from shapely.geometry import Point
from google.colab import files
import matplotlib.pyplot as plt
import statsmodels.api as sm

# --- 2. Load data centre CSV ---
df_data_centres = pd.read_csv("/combined_postcodes_geocoded.csv")

# Create geometry column
gdf_data_centres = gpd.GeoDataFrame(
    df_data_centres,
    geometry=gpd.points_from_xy(df_data_centres.lon, df_data_centres.lat),
    crs="EPSG:4326"
)

# --- 3. Load MSOA Excel data ---
excel_path = "/MSOA_meter_data.xlsx"  # upload your Excel file
df_energy = pd.read_excel(excel_path)

# --- 4. Load MSOA boundaries ---
msoa_path = "/MSOA_boundaries.gpkg"  # or shapefile
gdf_msoa = gpd.read_file(msoa_path)
gdf_msoa = gdf_msoa.to_crs("EPSG:4326")  # ensure same CRS

# --- 5. Merge energy data with boundaries ---
merged = gdf_msoa.merge(df_energy, left_on="MSOA21CD", right_on="MSOA code", how="inner")
merged = merged.dropna(subset=['geometry'])

# --- 6. Assign data centres to MSOAs ---
# Spatial join: which MSOA each data centre falls into
joined = gpd.sjoin(gdf_data_centres, merged, how="left", predicate="within")

# Count data centres per MSOA
data_centre_counts = joined.groupby('MSOA21CD').size()

# Merge count back into merged MSOA GeoDataFrame
merged['data_centre_count'] = merged['MSOA21CD'].map(data_centre_counts).fillna(0)

# --- 7. Compute correlation ---
correlation = merged['Mean \nconsumption\n(kWh per meter)'].corr(merged['data_centre_count'])
print(f"Correlation between electricity consumption and number of data centres: {correlation:.3f}")

# --- 8. Optional scatter plot ---
plt.figure(figsize=(8,6))
plt.scatter(merged['data_centre_count'], merged['Mean \nconsumption\n(kWh per meter)'], alpha=0.5)
plt.xlabel("Number of Data Centres in MSOA")
plt.ylabel("Mean Non-Domestic Electricity Consumption (kWh per meter)")
plt.title("Data Centres vs MSOA Electricity Consumption")
plt.show()


# --- 10. Create folium map ---
m = folium.Map(location=[54.5, -3], zoom_start=6, tiles="cartodb positron")

# Choropleth for electricity consumption
folium.Choropleth(
    geo_data=merged,
    data=merged,
    columns=["MSOA21CD", "Mean \nconsumption\n(kWh per meter)"],
    key_on="feature.properties.MSOA21CD",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Mean Non-Domestic Electricity Consumption (kWh) by MSOA"
).add_to(m)

# Overlay data centres
for idx, row in gdf_data_centres.iterrows():
    folium.CircleMarker(
        location=[row.lat, row.lon],
        radius=1,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        popup=row['postcode']
    ).add_to(m)

# Save and download map
m.save("msoa_consumption_map.html")
files.download("msoa_consumption_map.html")

# Display map inline
m


Output hidden; open in https://colab.research.google.com to view.